In [162]:
import re
import nltk
import string
import numpy as np
import networkx as nx
from nltk.cluster.util import cosine_distance
import heapq
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BASHA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BASHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [163]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
def preprocess(text):
    formatted_text = text.lower()
    tokens = []  # storing each words of sentence with out stop words
    for token in nltk.word_tokenize(formatted_text):
        tokens.append(token)
    
    tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
    formatted_text = ' '.join(element for element in tokens) # creating sentence without stop words and punctations
    
    return formatted_text

In [7]:
original_text = '''artificial intelligence is human like intelligence.
                   It is the study of intelligent artificial agents.
                   Science and engineering to produce intelligent machines.
                   Solve problems and have intelligence.
                   Related to intelligent behavior.
                   Developing of reasoning machines.
                   Learn from mistakes and successes.
                   Artificial intelligence is related to reasoning in everyday situations.'''
original_text =re.sub(r'\s+',' ',original_text) # \s fro space and + for one or more than one space
original_text

'artificial intelligence is human like intelligence. It is the study of intelligent artificial agents. Science and engineering to produce intelligent machines. Solve problems and have intelligence. Related to intelligent behavior. Developing of reasoning machines. Learn from mistakes and successes. Artificial intelligence is related to reasoning in everyday situations.'

In [62]:
original_sentences  = [sentence for sentence in nltk.sent_tokenize(original_text)]
formatted_sentnces  = [preprocess(original_sentence) for original_sentence in original_sentences]
formatted_sentnces

['artificial intelligence human like intelligence',
 'study intelligent artificial agents',
 'science engineering produce intelligent machines',
 'solve problems intelligence',
 'related intelligent behavior',
 'developing reasoning machines',
 'learn mistakes successes',
 'artificial intelligence related reasoning everyday situations']

In [177]:
def calculate_sentence_similarity(sentence1, sentence2):
    words1 = [word for word in nltk.word_tokenize(sentence1)]
    words2 = [word for word in nltk.word_tokenize(sentence2)]
    
    #print(words1)
    #print(words2)
    
    all_words = list(set(words1 + words2))
    #print(all_words)
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    for word in words1: # Bag of words
        vector1[all_words.index(word)] += 1
        
    for word in words2:
        vector2[all_words.index(word)] += 1
   # print(vector1)
   # print(vector2)
    return 1 - cosine_distance(vector1, vector2)

In [184]:
def calculate_similarity_matrix(sentences):
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
    return similarity_matrix        

In [185]:
calculate_similarity_matrix(formatted_sentnces)

array([[0.        , 0.18898224, 0.        , 0.43643578, 0.        ,
        0.        , 0.        , 0.46291005],
       [0.18898224, 0.        , 0.2236068 , 0.        , 0.28867513,
        0.        , 0.        , 0.20412415],
       [0.        , 0.2236068 , 0.        , 0.        , 0.25819889,
        0.25819889, 0.        , 0.        ],
       [0.43643578, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.23570226],
       [0.        , 0.28867513, 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.23570226],
       [0.        , 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.23570226],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.46291005, 0.20412415, 0.        , 0.23570226, 0.23570226,
        0.23570226, 0.        , 0.        ]])

In [189]:
def summarize(text, number_of_sentences,percentage = 0):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
    formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
    similarity_matrix = calculate_similarity_matrix(formatted_sentences)
    #print(similarity_matrix)

    similarity_graph = nx.from_numpy_array(similarity_matrix)
    #print(similarity_graph.nodes)
    #print(similarity_graph.edges)

    scores = nx.pagerank(similarity_graph)
    #print(scores)
    ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse=True)
    #print(ordered_scores)

    if percentage > 0:
        number_of_sentences = int(len(formatted_sentences) * percentage)

    best_sentences = []
    for sentence in range(number_of_sentences):
        best_sentences.append(ordered_scores[sentence][1])
    summary = " ".join(best_sentences)
    return summary

In [191]:
summarize(original_text, 3)

'Artificial intelligence is related to reasoning in everyday situations. artificial intelligence is human like intelligence. It is the study of intelligent artificial agents.'